In [2]:
%pip install --upgrade pip
%pip install google
%pip install --upgrade google-cloud-bigquery
%pip install --upgrade google-cloud-storage
%pip install --upgrade google-cloud-bigquery-storage
%pip install pandas
%pip install db-dtypes

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.



In [3]:
from google.cloud import bigquery
from google.cloud import storage
from google.oauth2 import service_account
import pandas as pd

# Clients


In [4]:
project_id = 'bigquery-midterm-exam'
dataset_id = 'datamodel_midterm_dataset'

In [5]:
# Initialize BigQuery client
credentials = service_account.Credentials.from_service_account_file(
    'bigquery-midterm-exam-5265cdf05f9b.json')

bigquery_client = bigquery.Client(project=project_id, credentials=credentials)
storage_client = storage.Client(project=project_id, credentials=credentials)

In [43]:
bucket_name = "datamodel_midterm"

In [6]:
customer_logs_path = 'customer_logs.csv'
ad_performance_path = 'ad_performance.csv'

In [46]:
# 上傳 customer_logs.csv 至 Google Cloud Storage
csv_file_path = 'customer_logs.csv'
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob('customer_logs.csv')
blob.upload_from_filename(csv_file_path)

# 將 CSV 資料作為外部表格載入 BigQuery
table_id = 'customer_logs'
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True
)
uri = f'gs://{bucket_name}/customer_logs.csv'

load_job = bigquery_client.load_table_from_uri(
    uri, f"{dataset_id}.{table_id}", job_config=job_config
)
load_job.result()  # 等待作業完成
print("上傳完成")

# 上傳 customer_logs.csv 至 Google Cloud Storage
csv_file_path = 'ad_performance.csv'
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob('ad_performance_logs.csv')
blob.upload_from_filename(csv_file_path)

# 將 CSV 資料作為外部表格載入 BigQuery
table_id = 'ad_performance_logs'
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True
)
uri = f'gs://{bucket_name}/ad_performance_logs.csv'

load_job = bigquery_client.load_table_from_uri(
    uri, f"{dataset_id}.{table_id}", job_config=job_config
)
load_job.result()  # 等待作業完成
print("上傳完成")

上傳完成
上傳完成


# Task 1


## 1.1 資料擷取與上傳


In [5]:
dataset_id = 'datamodel_midterm_dataset'
bucket_name = 'datamodel_midterm'

In [14]:
bucket = storage_client.bucket(bucket_name)

In [16]:
# Upload customer_logs.csv
customer_logs_blob = bucket.blob('customer_logs.csv')
customer_logs_blob.upload_from_filename('customer_logs.csv')

# Upload ad_performance_logs.csv
ad_performance_logs_blob = bucket.blob('ad_performance_logs.csv')
ad_performance_logs_blob.upload_from_filename('ad_performance.csv')

## 1.2 Transform Data


將score非0的資料取出來

In [9]:
# Normalize 'score' in Google Trends
normalize_query = f"""
CREATE OR REPLACE TABLE {project_id}.{dataset_id}.normalized_international_top_rising_terms AS
SELECT
  country_code,
  country_name,
  term,
  week,
  (score - MIN(score) OVER()) / (MAX(score) OVER() - MIN(score) OVER()) AS score
FROM
  bigquery-public-data.google_trends.international_top_rising_terms
WHERE score IS NOT null
"""
job = bigquery_client.query(normalize_query)
job.result()
print("Score normalization completed.")

Score normalization completed.


## 1.3 Integrate Datasets


In [21]:
# Normalize 'score' in Google Trends
integrated_query = f"""
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.combined_geotargets_criteria` AS
SELECT
    g.criteria_id,
    g.en_name AS geographic_target_name,
    g.country_code,
    m.target_city
FROM
    `bigquery-public-data.google_ads.geotargets` AS g
LEFT JOIN
    `bigquery-public-data.google_ads_geo_mapping_us.ads_geo_criteria_mapping` AS m
ON
    CAST(g.criteria_id AS INT64) = m.ads_criteria_id;
"""
job = bigquery_client.query(integrated_query)
job.result()
print("ads_geo_criteria_mapping and geotargets combined completed.")


ads_geo_criteria_mapping and geotargets combined completed.


將ga_sessions_20170801中pageviews和transactions的null值改為0並取出來

In [10]:
integrated_query = f"""
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.integrated_ad_analysis` AS
SELECT 
    trends.country_code, 
    trends.country_name, 
    trends.term, 
    trends.score,
    trends.week,
    ads.ad_id, 
    ads.impressions, 
    ads.clicks, 
    ads.conversions,
    logs.user_action,
    logs.timestamp,
    IFNULL(analytics.totals.pageviews, 0) AS pageviews, 
    IFNULL(analytics.totals.transactions, 0) AS transactions
FROM 
    `bigquery-midterm-exam.datamodel_midterm_dataset.normalized_international_top_rising_terms` AS trends
LEFT JOIN 
    `bigquery-midterm-exam.datamodel_midterm_dataset.customer_logs` AS logs
ON 
    trends.country_name = logs.country
LEFT JOIN 
    `bigquery-midterm-exam.datamodel_midterm_dataset.ad_performance_logs` AS ads
ON 
    trends.country_name = ads.country
LEFT JOIN 
    `bigquery-public-data.google_analytics_sample.ga_sessions_20170801` AS analytics
ON 
    trends.country_name = analytics.geoNetwork.country
WHERE 
    RAND() < (0.01)
"""
job = bigquery_client.query(integrated_query)
job.result()
print("integrated_query with NULL value handling completed.")

integrated_query with NULL value handling completed.


# Task 2

## Regional Trends

In [23]:
# SQL query for Regional Trends: Top 5 Terms by Country with corrected handling
query_top_terms_fixed = f"""
WITH RankedTerms AS (
    SELECT 
      country_name,
      term,
      score,
      RANK() OVER (PARTITION BY country_name ORDER BY score DESC) AS rank
    FROM 
      `{project_id}.{dataset_id}.integrated_ad_analysis`
)
SELECT 
  country_name,
  term,
  score,
  rank
FROM 
  RankedTerms
WHERE 
  rank <= 5;
"""

# Execute the corrected query
query_job_top_terms_fixed = bigquery_client.query(query_top_terms_fixed)

# Convert results to a Pandas DataFrame
df_top_terms_fixed = query_job_top_terms_fixed.to_dataframe()

# Display the results as a DataFrame
df_top_terms_fixed

,country_name,term,score,rank
0,Saudi Arabia,تشيلسي ضد آرسنال,1.0,1
1,Saudi Arabia,ترتيب الدوري السعودي,1.0,1
2,Saudi Arabia,مباراة المنتخب السعودي,1.0,1
3,Saudi Arabia,تصفيات كأس العالم,1.0,1
4,Saudi Arabia,باختاكور ضد العين,1.0,1
...,...,...,...,...
755817,Germany,1. fc magdeburg – hertha,1.0,1
755818,Germany,rb leipzig – mönchengladbach,1.0,1
755819,Germany,regenradar,1.0,1
755820,Germany,chase oliver,1.0,1


## Correlations

In [24]:
# SQL query for extended Correlation Analysis
query_correlation_analysis = f"""
WITH Correlations AS (
    SELECT 
        country_name,
        CORR(score, conversions) AS score_conversion_corr,
        CORR(score, pageviews) AS score_pageviews_corr,
        CORR(score, transactions) AS score_transactions_corr,
        CORR(conversions, pageviews) AS conversions_pageviews_corr,
        CORR(conversions, transactions) AS conversions_transactions_corr,
        CORR(pageviews, transactions) AS pageviews_transactions_corr
    FROM 
        `{project_id}.{dataset_id}.integrated_ad_analysis`
    GROUP BY 
        country_name
)
SELECT 
    country_name,
    score_conversion_corr,
    score_pageviews_corr,
    score_transactions_corr,
    conversions_pageviews_corr,
    conversions_transactions_corr,
    pageviews_transactions_corr
FROM 
    Correlations;
"""

# Execute the query for correlation analysis
query_job_correlation_analysis = bigquery_client.query(query_correlation_analysis)

# Convert results to a Pandas DataFrame
df_correlation_analysis = query_job_correlation_analysis.to_dataframe()

# Display the results
df_correlation_analysis


,country_name,score_conversion_corr,score_pageviews_corr,score_transactions_corr,conversions_pageviews_corr,conversions_transactions_corr,pageviews_transactions_corr
0,Brazil,NaN,0.001067,NaN,NaN,NaN,NaN
1,Taiwan,NaN,-0.003204,NaN,NaN,NaN,NaN
2,Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN
3,Japan,NaN,-0.001170,NaN,NaN,NaN,NaN
4,Canada,NaN,-0.000398,NaN,NaN,NaN,NaN
5,Mexico,NaN,0.000486,NaN,NaN,NaN,NaN
6,Sweden,NaN,NaN,NaN,NaN,NaN,NaN
7,Turkey,NaN,0.000064,NaN,NaN,NaN,NaN
8,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN
9,Indonesia,NaN,0.000104,NaN,NaN,NaN,NaN


## Ad Performance CTR

In [25]:
# SQL query for CTR Calculation
query_ctr_calculation = f"""
WITH CTRs AS (
    SELECT 
        ad_id,
        country_name,
        SAFE_DIVIDE(SUM(clicks), NULLIF(SUM(impressions), 0)) AS ctr
    FROM 
        `{project_id}.{dataset_id}.integrated_ad_analysis`
    GROUP BY
        ad_id, country_name
)
SELECT 
    ad_id,
    country_name,
    ctr
FROM 
    CTRs;
"""

# Execute the query for CTR calculation
query_job_ctr_calculation = bigquery_client.query(query_ctr_calculation)

# Convert results to a Pandas DataFrame
df_ctr_calculation = query_job_ctr_calculation.to_dataframe()
df_ctr_calculation

,ad_id,country_name,ctr
0,AD014,Japan,0.434123
1,AD010,Canada,0.166361
2,AD019,France,0.274062
3,AD001,Norway,3.030445
4,AD020,India,6.972141
5,AD015,Italy,0.272128
6,AD018,Brazil,7.482578
7,AD012,Poland,0.158819
8,None,Denmark,NaN
9,None,Finland,NaN


## Statistical Approximations

In [26]:
# SQL query for Statistical Approximations using APPROX_COUNT_DISTINCT
query_statistical_approx = f"""
SELECT 
    APPROX_COUNT_DISTINCT(country_name) AS unique_countries
FROM 
    `{project_id}.{dataset_id}.integrated_ad_analysis`;
"""

# Execute the query
query_job_statistical_approx = bigquery_client.query(query_statistical_approx)

# Convert results to a Pandas DataFrame
df_statistical_approx = query_job_statistical_approx.to_dataframe()

# Display the result as a DataFrame
df_statistical_approx

,unique_countries
0,42


# Task 3: Machine Learning with BigQueryML

## Feature Selection

In [6]:
# Feature Selection: Create a feature table
query_feature_table = f"""
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.feature_table` AS
SELECT 
    score,
    SAFE_DIVIDE(clicks, NULLIF(impressions, 0)) AS ctr,
    CASE WHEN user_action = 'click' THEN 1 ELSE 0 END AS user_action,
    conversions
FROM 
    `{project_id}.{dataset_id}.integrated_ad_analysis`
WHERE 
    conversions IS NOT NULL;
"""
feature_selection = bigquery_client.query(query_feature_table).result()

## Model Training

In [8]:
# Train the linear regression model
query_train_model = f"""
CREATE OR REPLACE MODEL `{project_id}.{dataset_id}.conversion_model`
OPTIONS(
    model_type = 'linear_reg',
    input_label_cols = ['conversions'],
    DATA_SPLIT_METHOD = 'RANDOM', -- Splits data randomly,
    DATA_SPLIT_EVAL_FRACTION = 0.2 -- Reserves 20% of data for testing
) AS
SELECT 
    score,
    ctr,
    user_action,
    conversions
FROM 
    `{project_id}.{dataset_id}.feature_table`;
"""
train_model = bigquery_client.query(query_train_model).result()

## Model Evaluation

In [9]:
# Evaluate the model
query_evaluate_model = f"""
SELECT
    *
FROM
    ML.EVALUATE(MODEL `{project_id}.{dataset_id}.conversion_model`, 
        (
        SELECT 
            score,
            ctr,
            user_action,
            conversions
        FROM 
            `{project_id}.{dataset_id}.feature_table`
        )
    );
"""
# Execute and retrieve evaluation results
evaluation_results = bigquery_client.query(query_evaluate_model).to_dataframe()
evaluation_results

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,284.506423,104263.086873,1.529384,300.772886,0.038325,0.038325
